In [42]:
import pennylane as qml
from pennylane import numpy as np

weights = np.array([2, 3, 4, 5])
values = np.array([3, 4, 5, 6])
weight_limit = 2
n_items = len(weights)
n_qubits = n_items

# Define the device
dev = qml.device('default.qubit', wires=n_qubits)


In [43]:
# flip the sign of the amplitude of states that satisfy the 
# weight constraint and have the highest value
def oracle():
    for i in range(n_qubits):
        qml.Hadamard(wires=i)

    # Apply multi-controlled Z gate to mark valid states
    marked_state = sum([weights[i] for i in range(n_qubits) if weights[i] <= weight_limit])
    for i in range(n_qubits):
        if weights[i] > weight_limit:
            qml.PauliX(wires=i)
    
    qml.MultiControlledX(control_wires=[i for i in range(1, n_qubits)], wires=0)
    
    for i in range(n_qubits):
        if weights[i] > weight_limit:
            qml.PauliX(wires=i)
    
    for i in range(n_qubits):
        qml.Hadamard(wires=i)


In [44]:
# inverts the amplitudes about the mean.
def diffusion():
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.PauliX(wires=i)
    
    qml.MultiControlledX(control_wires=list(range(1, n_qubits)), wires=0)
    
    for i in range(n_qubits):
        qml.PauliX(wires=i)
        qml.Hadamard(wires=i)

In [45]:
@qml.qnode(dev)
def grover_circuit():
    # Apply Hadamard gates to all qubits to create superposition
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
    
    # Apply Grover iterations
    num_iterations = int(np.pi / 4 * np.sqrt(2**n_qubits))
    for _ in range(num_iterations):
        oracle()
        diffusion()
    
    return qml.probs(wires=range(n_qubits))


In [46]:
# Execute the circuit
probs = grover_circuit()
print(probs)

# Determine the most likely state
most_likely_state = np.argmax(probs)
print(f'Most likely state: {bin(most_likely_state)[2:].zfill(n_qubits)} with probability {probs[most_likely_state]}')


[0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625
 0.0625 0.0625 0.0625 0.0625 0.0625 0.0625]
Most likely state: 0000 with probability 0.062499999999999487


In [48]:
import pennylane as qml
from pennylane import numpy as np

# Define problem parameters
values = np.array([3, 2, 1])
weights = np.array([2, 1, 1])
capacity = 3
n_items = len(values)

# Initialize the device
dev = qml.device('default.qubit', wires=n_items)

def knapsack_oracle():
    """Oracle that flips the sign of the state if it corresponds to a valid solution."""
    @qml.qnode(dev)
    def oracle():
        for i in range(n_items):
            qml.Hadamard(wires=i)
        
        # Check for valid solutions
        for i in range(n_items):
            if weights[i] > capacity:
                qml.PauliX(wires=i)
        
        # Apply multi-controlled-Z gate to mark the valid solution
        qml.MultiControlledX(control_wires=range(n_items), wires=0, work_wires=range(1, n_items))
        
        for i in range(n_items):
            if weights[i] > capacity:
                qml.PauliX(wires=i)
        
        for i in range(n_items):
            qml.Hadamard(wires=i)
        
        return qml.state()
    return oracle

def diffusion_operator():
    """Diffusion operator (inversion about the mean)."""
    @qml.qnode(dev)
    def diffusion():
        for i in range(n_items):
            qml.Hadamard(wires=i)
            qml.PauliX(wires=i)
        
        qml.MultiControlledX(control_wires=range(n_items-1), wires=n_items-1)
        
        for i in range(n_items):
            qml.PauliX(wires=i)
            qml.Hadamard(wires=i)
        
        return qml.state()
    return diffusion

# Apply Grover's algorithm
oracle = knapsack_oracle()
diffusion = diffusion_operator()

@qml.qnode(dev)
def grover():
    # Apply Hadamard gates to create a superposition
    for i in range(n_items):
        qml.Hadamard(wires=i)
    
    # Apply Grover iterations
    for _ in range(int(np.sqrt(2**n_items))):
        oracle()
        diffusion()
    
    return qml.probs(wires=range(n_items))

# Execute the algorithm
probabilities = grover()
print("Probabilities of different states:", probabilities)


ValueError: The control wires must be different from the base operation wires.

In [49]:
import pennylane as qml
from pennylane import numpy as np

# Define problem parameters
values = np.array([3, 2, 1])
weights = np.array([2, 1, 1])
capacity = 3
n_items = len(values)

# Initialize the device
dev = qml.device('default.qubit', wires=n_items + 1)  # Add an extra wire for the auxiliary qubit

def knapsack_oracle():
    """Oracle that flips the sign of the state if it corresponds to a valid solution."""
    @qml.qnode(dev)
    def oracle():
        # Apply Hadamard gates
        for i in range(n_items):
            qml.Hadamard(wires=i)
        
        # Compute the total weight
        for i in range(n_items):
            if weights[i] > 0:
                qml.CNOT(wires=[i, n_items])
        
        # Flip the state of the auxiliary qubit if the total weight is within capacity
        qml.PauliX(wires=n_items)
        qml.CZ(wires=[0, n_items])
        qml.PauliX(wires=n_items)
        
        # Uncompute the total weight
        for i in range(n_items):
            if weights[i] > 0:
                qml.CNOT(wires=[i, n_items])
        
        return qml.state()
    return oracle

def diffusion_operator():
    """Diffusion operator (inversion about the mean)."""
    @qml.qnode(dev)
    def diffusion():
        # Apply Hadamard and Pauli-X gates
        for i in range(n_items):
            qml.Hadamard(wires=i)
            qml.PauliX(wires=i)
        
        # Multi-controlled Z gate
        qml.MultiControlledX(control_wires=range(n_items - 1), wires=n_items - 1)
        
        # Apply Pauli-X and Hadamard gates
        for i in range(n_items):
            qml.PauliX(wires=i)
            qml.Hadamard(wires=i)
        
        return qml.state()
    return diffusion

# Apply Grover's algorithm
oracle = knapsack_oracle()
diffusion = diffusion_operator()

@qml.qnode(dev)
def grover():
    # Apply Hadamard gates to create a superposition
    for i in range(n_items):
        qml.Hadamard(wires=i)
    
    # Apply Grover iterations
    num_iterations = int(np.sqrt(2**n_items))
    for _ in range(num_iterations):
        oracle()
        diffusion()
    
    return qml.probs(wires=range(n_items))

# Execute the algorithm
probabilities = grover()
print("Probabilities of different states:", probabilities)


Probabilities of different states: [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]


In [50]:
import pennylane as qml
from pennylane import numpy as np

# Define problem parameters
values = np.array([3, 2, 1])
weights = np.array([2, 1, 1])
capacity = 3
n_items = len(values)

# Initialize the device
dev = qml.device('default.qubit', wires=n_items + 1)  # Add an extra wire for the auxiliary qubit

def knapsack_oracle():
    """Oracle that flips the sign of the state if it corresponds to a valid solution."""
    @qml.qnode(dev)
    def oracle():
        # Apply Hadamard gates to the first n_items qubits
        for i in range(n_items):
            qml.Hadamard(wires=i)
        
        # Mark valid states
        for i in range(n_items):
            if weights[i] > 0:
                qml.CNOT(wires=[i, n_items])
        
        # Apply phase flip if the total weight is within capacity
        qml.PauliX(wires=n_items)
        qml.MultiControlledX(control_wires=range(n_items), wires=n_items)
        qml.PauliX(wires=n_items)
        
        # Uncompute the total weight calculation
        for i in range(n_items):
            if weights[i] > 0:
                qml.CNOT(wires=[i, n_items])
        
        return qml.state()
    return oracle

def diffusion_operator():
    """Diffusion operator (inversion about the mean)."""
    @qml.qnode(dev)
    def diffusion():
        for i in range(n_items):
            qml.Hadamard(wires=i)
            qml.PauliX(wires=i)
        
        qml.MultiControlledX(control_wires=range(n_items - 1), wires=n_items - 1)
        
        for i in range(n_items):
            qml.PauliX(wires=i)
            qml.Hadamard(wires=i)
        
        return qml.state()
    return diffusion

@qml.qnode(dev)
def grover():
    # Apply Hadamard gates to create a superposition
    for i in range(n_items):
        qml.Hadamard(wires=i)
    
    # Apply Grover iterations
    num_iterations = int(np.sqrt(2**n_items))
    for _ in range(num_iterations):
        oracle()
        diffusion()
    
    return qml.probs(wires=range(n_items))

# Execute the algorithm
probabilities = grover()
print("Probabilities of different states:", probabilities)


Probabilities of different states: [0.125 0.125 0.125 0.125 0.125 0.125 0.125 0.125]
